In [1]:
from random import choices

import networkx as nx
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from models.observableSubgraph import create_subgraph, simulate_observable_nodes

In [3]:
from models.seeding import Seeding

In [11]:
from models.independentCascade import IndependentCascade

In [4]:
file = open('data/weic-graphdata/hep.txt', 'r')
first_line = file.readline()
meta = first_line.split()
num_nodes = int(meta[0])
num_edges = int(meta[1])
print(f"|V|: {num_nodes}, |E|: {num_edges}")

edges = [line.strip().split() for line in file.readlines()]
edges = [(int(e[0]), int(e[1])) for e in edges]

|V|: 15233, |E|: 58891


In [5]:
graph = nx.DiGraph()
graph.add_edges_from(edges)
graph.number_of_nodes(), graph.number_of_edges() # 32K edges match the paper
# duplicated edges

(15233, 32235)

In [6]:
target_visibility = 0.1
fully_observ_O, boundary_nodes = simulate_observable_nodes(graph, target_visibility)
observable_nodes = fully_observ_O + boundary_nodes
observable_graph = create_subgraph(graph, observable_nodes)
# seeds = seed_weighted_degree(observable_graph, weight = 3, num_seed = 5)
# activated_nodes = simulate_weighted_IC(graph, seeds)
# print(f"spread: {len(activated_nodes)}")

In [8]:
seeding = Seeding(observable_graph, method = 'weighted-degree-based')
seeds = seeding(boundary_nodes, weight = 3, num_seed = 5)
seeds

[196, 66, 474, 14, 239]

In [8]:
seeding = Seeding(observable_graph, method = 'random')
seeds = seeding(num_seed = 5)
seeds

[885, 13094, 539, 6097, 8879]

In [91]:
seeds = seeding(num_seed = 5, activate_neighbor = True)
seeds

[32, 111, 1643, 462, 1441]

In [250]:
ic_model = IndependentCascade(graph, seeds, weighted = False)
activated = ic_model()
len(activated)

17